Main notebook :3

In [ ]:
from utils import get_image, save_image, show_image, remove_small_islands, get_corners, normalize_page

In [ ]:
PATH_TEMPLATE = "book1/family-tree-{}.png"
idx = -1
for i in range(4, 13):
    print(f"i: {i}")
    filepath = PATH_TEMPLATE.format(f"{i:02d}")
    a = get_image(filepath)
    a = remove_small_islands(a)
    rows, cols = a.shape
    # Find starting pixel of page border
    i = int(rows / 2)
    j = cols - 20
    while a[i][j] or a[i][j - 1] or a[i][j - 2] or a[i][j - 3]:
        j -= 1
    right_page_corners = get_corners(a, i, j)
    print("right_page_corners", right_page_corners)
    right_page = normalize_page(a, right_page_corners)
    save_image(right_page, f"book1/pages/{idx}.png")
    idx += 1

    i = int(rows / 2)
    j = 0
    while a[i][j] or a[i][j + 1] or a[i][j + 2] or a[i][j + 3]:
        j += 1
    left_page_corners = get_corners(a, i, j)
    print("left_page_corners", left_page_corners)
    left_page = normalize_page(a, left_page_corners)
    save_image(left_page, f"book1/pages/{idx}.png")
    idx += 1


# Crop

In [ ]:
from utils import trim_borders, shrink_page

PATH_TEMPLATE = "book1/pages/{}.png"
for i in range(17):
    print(f"i: {i}")
    filepath = PATH_TEMPLATE.format(i)
    a = get_image(filepath)
    a = trim_borders(a)
    a = shrink_page(a)
    save_image(a, f"book1/graphs/{i}.png")

In [ ]:
from utils import trim_borders, shrink_page, is_tree_start_page, merge_graphs

In [7]:
PATH_TEMPLATE = "book1/pages/{}.png"
pages = []
is_page_tree_start = []
for i in range(17):
    filepath = PATH_TEMPLATE.format(i)
    a = get_image(filepath)
    a = trim_borders(a)
    is_page_tree_start.append(is_tree_start_page(a))
    a = shrink_page(a)
    pages.append(a)
#   save_image(a, f"book1/graphs/{i}.png")

print("Merging and saving graphs")
i = 0
while i < 17:
    start_i = i
    graph = pages[i]
    i += 1
    while i < 17 and not is_page_tree_start[i]:
        print(f"---------Merge starting at i: {i}")
        left = pages[i]
        graph = merge_graphs(left, graph)
        i += 1
    save_image(graph, f"book1/graphs/{start_i}_{i-1}.png")


Merging and saving graphs
---------Merge starting at i: 14
Left x, right x
[537] [548]
Adjusted left x, right x
[548] [548]
uint8 uint8
---------Merge starting at i: 15
Left x, right x
[863, 1237, 1523] [860, 1245, 1536]
Adjusted left x, right x
[863, 1237, 1523] [863, 1248, 1539]
uint8 uint8
---------Merge starting at i: 16
Left x, right x
[884] [863, 1236]
Adjusted left x, right x
[884] [884, 1257]
!!!!!!!!!!!!!!!!!!Orphans [1257] on right side
uint8 uint8


# Graph parsing heuristic

In [43]:
def find_lines(image, threshold=70):
  """
  Find connected components in a binary image that span more than 70 pixels
  in either width or height.
  
  Args:
    image: 2D list where 0 = black pixel, 1 = white pixel
    
  Returns:
    List of sets, where each set contains (row, col) tuples of pixels
    in a large connected component
  """
  if len(image) == 0 or len(image[0]) == 0:
    return []
  
  image = image.copy()
  rows = len(image)
  cols = len(image[0])
  results = []
  
  # Iterate through entire array
  for i in range(rows):
    for j in range(cols):
      # If we find a black pixel (0)
      if image[i][j] == 0:
        # BFS to find connected component
        component = set()
        queue = [(i, j)]
        image[i][j] = 1  # Mark as visited
        
        min_x = max_x = i
        min_y = max_y = j
        
        while queue:
          x, y = queue.pop(0)
          component.add((x, y))
          
          # Update bounds
          min_x = min(min_x, x)
          max_x = max(max_x, x)
          min_y = min(min_y, y)
          max_y = max(max_y, y)
          
          # Check four direct directions
          for dx, dy in [(0, 1), (0, -1), (1, 0), (-1, 0)]:
            nx, ny = x + dx, y + dy
            
            # Check bounds and if pixel is black
            if (0 <= nx < rows and 0 <= ny < cols and 
              image[nx][ny] == 0):
              image[nx][ny] = 1  # Mark as visited
              queue.append((nx, ny))
        
        # Check if component is large enough
        if max_x - min_x > threshold or max_y - min_y > threshold:
          results.append(component)
  
  return results

def find_line_ends(points, threshold=50):
  """
  Find the endpoints of a line represented by a set of points.
  
  Args:
    points: Set of (x, y) tuples representing a connected component
    
  Returns:
    Tuple of two sets: (left_end_ys, right_end_ys)
    Each set contains y-coordinates of endpoints, with adjacent values removed
  """
  if not points:
    return (set(), set())
  
  # Find minimum x coordinate
  min_x = min(x for x, y in points)
  
  # Get y coordinates at the top
  top_ys = {y for x, y in points if x == min_x}
  
  # Remove adjacent numbers
  top_ys_filtered = remove_adjacent(top_ys)
  
  # Find maximum x coordinate
  max_x = max(x for x, y in points)
  
  # Get y coordinates near the bottom
  bottom_ys = {y for x, y in points if abs(x - max_x) <= threshold}
  
  # Remove adjacent numbers
  bottom_ys_filtered = remove_adjacent(bottom_ys)
  
  top_points = [(min_x, y) for y in sorted(top_ys_filtered)]
  bottom_points = [(max_x, y) for y in sorted(bottom_ys_filtered)]
  return (top_points, bottom_points)


def remove_adjacent(numbers, threshold=30):
  """
  Remove adjacent numbers from a set, keeping only the first of each sequence.
  For example: {4, 6, 7, 8} becomes {4, 7}
  
  Args:
    numbers: Set of integers
    
  Returns:
    Set with adjacent numbers removed
  """
  if not numbers:
    return set()
  
  sorted_nums = sorted(numbers)
  result = {sorted_nums[0]}
  
  for i in range(1, len(sorted_nums)):
    # If current number is not adjacent to previous, keep it
    if sorted_nums[i] - sorted_nums[i-1] > threshold:
      result.add(sorted_nums[i])
  
  return result


In [44]:
a = get_image("book1/graphs/13_16.png")
# a = get_image("book1/cropped/13.png")
raw_results = find_lines(a)
results = []
for raw_result in raw_results:
  parents, children = find_line_ends(raw_result)
  if len(parents) != 1:
    raise ValueError("More than one parent found", parents)
  parent = parents[0]
  results.append((parent, children))
  print(parent, children)

(158, 3525) [(395, 3525)]
(474, 3526) [(708, 2572), (708, 3525)]
(786, 2569) [(1026, 135), (1026, 343), (1026, 2368), (1026, 2568)]
(787, 3525) [(1021, 3220), (1021, 3525)]
(1116, 342) [(1358, 237), (1358, 341)]
(1117, 131) [(1358, 28), (1358, 131)]
(1155, 2570) [(1348, 2466), (1348, 2569)]
(1207, 3216) [(1344, 2709), (1344, 2809), (1344, 2909), (1344, 3215)]
(1207, 3530) [(1344, 3530)]
(1211, 2362) [(1350, 542), (1350, 850), (1350, 1157), (1350, 1362), (1350, 1639), (1350, 1851), (1350, 2158), (1350, 2361)]
(1462, 3217) [(1652, 3018), (1652, 3116), (1652, 3217)]
(1465, 849) [(1655, 642), (1655, 746), (1655, 849)]
(1466, 1161) [(1655, 953), (1655, 1058), (1655, 1160)]
(1466, 1365) [(1655, 1262), (1655, 1365)]
(1471, 2809) [(1658, 2808)]
(1471, 2909) [(1659, 2908)]
(1480, 1642) [(1670, 1467), (1670, 1642)]
(1480, 1848) [(1670, 1746), (1670, 1848)]
(1480, 2149) [(1671, 1952), (1671, 2047), (1671, 2149)]
(1480, 2361) [(1671, 2258), (1671, 2360)]
(1480, 2465) [(1670, 2465)]
(1525, 3530) [(

In [45]:
import dataclasses
from typing import List, Optional, Tuple

@dataclasses.dataclass
class Node:
  top: Optional[Tuple[int, int]] = None
  bot: Optional[Tuple[int, int]] = None
  children: List[Tuple[int, int]] = dataclasses.field(default_factory=list)
  def __str__(self):
    return f"Node(top={self.top}, bot={self.bot})"

nodes = []
for parent, children in results:
  pn = Node(bot=parent)
  nodes.append(pn)
  for c in children:
    cn = Node(top=c)
    pn.children.append(cn)
    nodes.append(cn)

while True:
  made_progress = False
  for i in range(len(nodes)):
    for j in range(len(nodes)):
      if i == j:
        continue
      p = nodes[i]
      c = nodes[j]
      # print(p, c)
      # Only try the connection is p is missing bot and c is missing top
      if p.bot or c.top:
        continue
      # print("valid")
      tx, ty = p.top
      bx, by = c.bot
      if 0 < bx - tx < 200 and abs(by - ty) < 20:
        # print("merge")
        # Same node, merge
        p.bot = c.bot
        p.children = c.children
        del nodes[j]
        made_progress = True
        break
    if made_progress:
      break
  if not made_progress:
    break

print(len(nodes))


55


In [46]:
def infer_ends(nodes, a):
  for n in nodes:
    if n.top is None:
      top = max(0, n.bot[0] - 200)
      y = n.bot[1]
      min_y = max(0, y - 30)
      max_y = min(a.shape[1] - 1, y + 30)
      while top < n.bot[0] and 0 not in a[top][min_y:max_y]:
        top += 1
      top = max(0, top - 10)
      n.top = (top, y)
    if n.bot is None:
      bot = min(a.shape[0] - 1, n.top[0] + 200)
      print('start bot', bot)
      y = n.top[1]
      min_y = max(0, y - 30)
      max_y = min(a.shape[1] - 1, y + 30)
      while bot > n.top[0] and 0 not in a[bot][min_y:max_y]:
        bot -= 1
      bot = min(bot + 10, a.shape[0] - 1)
      n.bot = (bot, y)

In [47]:
def verify_nodes(nodes):
  for n in nodes:
    if not (60 < n.bot[0] - n.top[0] < 250):
        print("node looks sus:", str(n))

In [48]:
infer_ends(nodes, a)
verify_nodes(nodes)

start bot 1558
start bot 1558
start bot 1558
start bot 1558
start bot 1548
start bot 1544
start bot 1550
start bot 1835
start bot 1835
start bot 1835
start bot 1835
start bot 1835
start bot 1835
start bot 1835
start bot 1835
start bot 1835
start bot 1835
start bot 1835
start bot 1835
start bot 1835
start bot 1835
start bot 1835
start bot 1835
start bot 1835
start bot 1835
start bot 1835
start bot 1835
start bot 1835
start bot 1835
start bot 1835
start bot 1835
start bot 1835
start bot 1835


In [49]:
print(nodes)

[Node(top=(81, 3525), bot=(158, 3525), children=[Node(top=(395, 3525), bot=(474, 3526), children=[Node(top=(708, 2572), bot=(786, 2569), children=[Node(top=(1026, 135), bot=(1117, 131), children=[Node(top=(1358, 28), bot=(1478, 28), children=[]), Node(top=(1358, 131), bot=(1478, 131), children=[])]), Node(top=(1026, 343), bot=(1116, 342), children=[Node(top=(1358, 237), bot=(1477, 237), children=[]), Node(top=(1358, 341), bot=(1476, 341), children=[])]), Node(top=(1026, 2368), bot=(1211, 2362), children=[Node(top=(1350, 542), bot=(1457, 542), children=[]), Node(top=(1350, 850), bot=(1465, 849), children=[Node(top=(1655, 642), bot=(1775, 642), children=[]), Node(top=(1655, 746), bot=(1773, 746), children=[]), Node(top=(1655, 849), bot=(1774, 849), children=[])]), Node(top=(1350, 1157), bot=(1466, 1161), children=[Node(top=(1655, 953), bot=(1776, 953), children=[]), Node(top=(1655, 1058), bot=(1777, 1058), children=[]), Node(top=(1655, 1160), bot=(1776, 1160), children=[])]), Node(top=(1

In [50]:
def print_trees(nodes):
  def print_tree(n, indent):
    print("  " * indent + str(n))
    for c in n.children:
      print_tree(c, indent + 1)

  children = []
  for n in nodes:
    for c in n.children:
      children.append(c)
  for n in nodes:
    if n not in children:
      print_tree(n, 0)

In [51]:
print_trees(nodes)

Node(top=(81, 3525), bot=(158, 3525))
  Node(top=(395, 3525), bot=(474, 3526))
    Node(top=(708, 2572), bot=(786, 2569))
      Node(top=(1026, 135), bot=(1117, 131))
        Node(top=(1358, 28), bot=(1478, 28))
        Node(top=(1358, 131), bot=(1478, 131))
      Node(top=(1026, 343), bot=(1116, 342))
        Node(top=(1358, 237), bot=(1477, 237))
        Node(top=(1358, 341), bot=(1476, 341))
      Node(top=(1026, 2368), bot=(1211, 2362))
        Node(top=(1350, 542), bot=(1457, 542))
        Node(top=(1350, 850), bot=(1465, 849))
          Node(top=(1655, 642), bot=(1775, 642))
          Node(top=(1655, 746), bot=(1773, 746))
          Node(top=(1655, 849), bot=(1774, 849))
        Node(top=(1350, 1157), bot=(1466, 1161))
          Node(top=(1655, 953), bot=(1776, 953))
          Node(top=(1655, 1058), bot=(1777, 1058))
          Node(top=(1655, 1160), bot=(1776, 1160))
        Node(top=(1350, 1362), bot=(1466, 1365))
          Node(top=(1655, 1262), bot=(1777, 1262))
          Node

In [52]:
import numpy as np
def get_name_image(node, a):
  y = node.top[1]
  name = a[node.top[0]+5:node.bot[0]-5, max(0, y-40):min(a.shape[1], y+40)]

  col_present = np.sum(1 - name, axis=0)
  min_y = 0
  while min_y < len(col_present) and not col_present[min_y]:
    min_y += 1
  min_y = max(min_y - 10, 0)
  max_y = len(col_present) - 1
  while max_y > 0 and not col_present[max_y]:
    max_y -= 1
  max_y = min(max_y + 10, len(col_present) - 1)
  name = name[:, min_y:max_y]
  print("min_y", min_y, "max_y", max_y)

  row_present = np.sum(1 - name, axis=1)
  min_x = 0
  while min_x < len(row_present) and not row_present[min_x]:
    min_x += 1
  min_x = max(min_x - 10, 0)
  max_x = len(row_present) - 1
  while max_x > 0 and not row_present[max_x]:
    max_x -= 1
  max_x = min(max_x + 10, len(row_present) - 1)
  name = name[min_x:max_x, :]
  print("min_x", min_x, "max_x", max_x)
  return name


In [53]:
import json

def sort_nodes(nodes: List[Node]) -> List[Node]:
    def compare_key(node):
        return (node.top[0], node.top[1])
    
    # First, sort by both top[0] and top[1] as a baseline
    nodes_sorted = sorted(nodes, key=compare_key)
    
    # Now apply the custom logic
    result = []
    i = 0
    while i < len(nodes_sorted):
        # Collect nodes that should be grouped together
        group = [nodes_sorted[i]]
        j = i + 1
        
        while j < len(nodes_sorted):
            # Check if top[0] values are within 60 of each other
            if abs(nodes_sorted[j].top[0] - nodes_sorted[i].top[0]) < 60:
                group.append(nodes_sorted[j])
                j += 1
            else:
                break
        
        # Sort the group by top[1]
        group.sort(key=lambda node: node.top[1])
        result.extend(group)
        i = j
    
    return result

nodes = sort_nodes(nodes)

idx = 0
tree = {}
node_to_idx = {(n.top, n.bot): idx for idx, n in enumerate(nodes)}
for n in nodes:
  print(n)
  tree[node_to_idx[(n.top, n.bot)]] = [node_to_idx[(c.top, c.bot)] for c in n.children]
  name_img = get_name_image(n, a)
  save_image(name_img, f"book1/names/13_{idx}.png")
  idx += 1
json.dump(tree, open("book1/names/13_tree.json", "w"))

Node(top=(81, 3525), bot=(158, 3525))
min_y 9 max_y 79
min_x 0 max_x 61
Node(top=(395, 3525), bot=(474, 3526))
min_y 14 max_y 77
min_x 0 max_x 65
Node(top=(708, 2572), bot=(786, 2569))
min_y 7 max_y 70
min_x 3 max_x 64
Node(top=(708, 3525), bot=(787, 3525))
min_y 16 max_y 78
min_x 7 max_x 61
Node(top=(1026, 135), bot=(1117, 131))
min_y 10 max_y 77
min_x 4 max_x 72
Node(top=(1026, 343), bot=(1116, 342))
min_y 12 max_y 74
min_x 4 max_x 70
Node(top=(1026, 2368), bot=(1211, 2362))
min_y 2 max_y 69
min_x 0 max_x 174
Node(top=(1026, 2568), bot=(1155, 2570))
min_y 8 max_y 77
min_x 0 max_x 118
Node(top=(1021, 3220), bot=(1207, 3216))
min_y 8 max_y 71
min_x 2 max_x 173
Node(top=(1021, 3525), bot=(1207, 3530))
min_y 12 max_y 77
min_x 2 max_x 173
Node(top=(1358, 28), bot=(1478, 28))
min_y 1 max_y 67
min_x 0 max_x 109
Node(top=(1358, 131), bot=(1478, 131))
min_y 14 max_y 79
min_x 0 max_x 109
Node(top=(1358, 237), bot=(1477, 237))
min_y 9 max_y 79
min_x 0 max_x 108
Node(top=(1358, 341), bot=(1476, 

In [ ]:
# TODO: Merge adjacent pages if their lines go off the edge (and if they're missing the tag). Warn on orphans

In [ ]:
13_37.png looks sus

# Graph parsing opencv

In [ ]:
import cv2
import math

img = get_image("book1/cropped/0.png")
lines = cv2.HoughLinesP(
    1-img,
    rho=1,
    theta=math.pi/180,
    threshold=10,
    minLineLength=60,
    maxLineGap=10
)

In [ ]:
lines = lines.squeeze().tolist()
# Swap coordinates if x1+y1 > x2+y2 to ensure consistent ordering
for line in lines:
    line[0], line[1], line[2], line[3] = line[1], line[0], line[3], line[2]
    if line[0] + line[1] > line[2] + line[3]:
        line[0], line[1], line[2], line[3] = line[2], line[3], line[0], line[1]

lines = sorted(lines, key=lambda x: x[0])

def deduplicate_lines(lines, threshold=10):
  """
  Deduplicate a 2D list where entries are considered duplicates
  if all corresponding values are within the threshold.
  """
  if not lines:
    return []
  
  deduplicated = []
  
  for line in lines:
    is_duplicate = False
    for existing in deduplicated:
      # Check if all corresponding values are within threshold
      if all(abs(line[i] - existing[i]) <= threshold for i in range(len(line))):
        is_duplicate = True
        break
    
    if not is_duplicate:
      deduplicated.append(line)
  
  return deduplicated

lines = deduplicate_lines(lines)
for line in lines:
  line2 = [line[1], line[0], line[3], line[2]]
  print(line2)

In [ ]:
print(lines.shape)

In [ ]:
for (x1, y1, x2, y2) in lines[:, 0]:
    cv2.line(img, (x1, y1), (x2, y2), (0,0,255), 1)

In [ ]:
show_image(1-img)

In [ ]:
print(stats)
print(centroids)
print(num_labels)
print(labels)

In [ ]:
for i, stat in enumerate(stats):
    x, y, w, h, area = stat
    if area > 50:  # adjust threshold
        name_img = img[y:y+h, x:x+w]
        save_image(name_img, f"names/name_{i}.png")

# attempt 2

In [ ]:
img = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)

# Optional blur to reduce noise
blur = cv2.GaussianBlur(img, (5, 5), 0)

# Binary inverse threshold — black border becomes white
_, thresh = cv2.threshold(blur, 50, 255, cv2.THRESH_BINARY_INV)

# Find contours
contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Sort contours by area (largest first)
contours = sorted(contours, key=cv2.contourArea, reverse=True)

In [ ]:
print(len(contours))

In [ ]:
page_corners = []

for cnt in contours[:2]:  # take top 2 (two pages)
    # Approximate contour to polygon
    peri = cv2.arcLength(cnt, True)
    approx = cv2.approxPolyDP(cnt, 0.02 * peri, True)
    print(approx)
    print(len(approx))

    if len(approx) == 4:
        corners = approx.reshape(4, 2)
        page_corners.append(corners)

# Visualize results
vis = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
for corners in page_corners:
    for x, y in corners:
        cv2.circle(vis, (int(x), int(y)), 10, (0, 0, 255), -1)

cv2.imwrite("detected_corners.png", vis)


# Sort each corner set to TL, TR, BR, BL order
def order_points(pts):
    s = pts.sum(axis=1)
    diff = np.diff(pts, axis=1)
    tl = pts[np.argmin(s)]
    br = pts[np.argmax(s)]
    tr = pts[np.argmin(diff)]
    bl = pts[np.argmax(diff)]
    return np.array([tl, tr, br, bl], dtype=np.float32)


ordered_pages = [order_points(c) for c in page_corners]

for i, corners in enumerate(ordered_pages):
    print(f"Page {i + 1} corners (TL, TR, BR, BL):\n", corners)

In [ ]:
print(ordered_pages)

# CSV Parser

In [ ]:
import pandas as pd

# Read the CSV file into a DataFrame
df = pd.read_csv("data/zeng_google_sheet.csv", keep_default_na=False)
df["id"] = df["id"].astype(int)
df["name_images"] = [[]] * len(df)
df["generation"] = df["generation"].astype(int)
df["parent"] = df["parent"].astype(int)
df["children"] = df["children"].apply(
    lambda x: [int(child) for child in x.split(",")] if x else []
)

# Iterate through rows and set parent based on children relationships
for _, row in df.iterrows():
    # Get the current node's ID and its children
    node_id = row["id"]
    children = row["children"]

    # For each child, set its parent to the current node_id
    for child_id in children:
        df.loc[df["id"] == child_id, "parent"] = node_id
df

In [ ]:
# Export DataFrame to JSONL format
df.to_json("data/book1.jsonl", orient="records", lines=True, force_ascii=False)

In [ ]:
import json

with open("data/book1.jsonl", "r") as f:
    records = [json.loads(line) for line in f]

print(records[0])
